# In Financial World, Analyst generally used MS Excell to calculate calculating principal and interest portion of installment using PPMT, IPMT functions, doing the same using popular data science programming languages Python

In [1]:
#Monthly instalment is calculated based on the following formula assuming constant payment and constant interest rate
#Ajeet Pandey
#PMT  = (rate*(fv+pv*(1+ rate)^nper))/((1+rate*type)*(1-(1+ rate)^nper))
#rate = Interest rate per month
#fv   = future value after the full loan is repaid.
#pv   = loan amount
#nper = Number of payments for loan
#type=0 = Payments are due at the end of the period. type = 1 Payments are due at the beginning of the period


def PMT(rate, nper,pv, fv=0, type=0):
    if rate!=0:
               pmt = (rate*(fv+pv*(1+ rate)**nper))/((1+rate*type)*(1-(1+ rate)**nper))
    else:
               pmt = (-1*(fv+pv)/nper)  
    return(pmt)


#Interest portion of monthly instalment 
#Calculation behind this function is dependent on PMT function.
#per = nth period
def IPMT(rate, per, nper,pv, fv=0, type=0):
  ipmt = -( ((1+rate)**(per-1)) * (pv*rate + PMT(rate, nper,pv, fv=0, type=0)) - PMT(rate, nper,pv, fv=0, type=0))
  return(ipmt)


#PPMT function returns principal portion of instalment.Difference between instalment amount and interest amount.
def PPMT(rate, per, nper,pv, fv=0, type=0):
  ppmt = PMT(rate, nper,pv, fv=0, type=0) - IPMT(rate, per, nper, pv, fv=0, type=0)
  return(ppmt)

In [2]:
#To calculate interest and principal amount of instalment of each period, 
#we need to loop PPMT and IPMT functions over sequence of periods of loan payment.

import numpy as np
import pandas as pd

def amortisation_schedule(amount, annualinterestrate, paymentsperyear, years):

    df = pd.DataFrame({'Principal' :[PPMT(annualinterestrate/paymentsperyear, i+1, paymentsperyear*years, amount)
                                     for i in range(paymentsperyear*years)],
                        'Interest' :[IPMT(annualinterestrate/paymentsperyear, i+1, paymentsperyear*years, amount)
                                              for i in range(paymentsperyear*years)]
                      }
                     )
    
    df['Instalment'] = df.Principal + df.Interest
    df['Balance'] = amount + np.cumsum(df.Principal) #Return the cumulative sum
    return(df)


In [3]:
df = amortisation_schedule(400000, 0.1055, 12, 4)

In [4]:
df.head(5)

,Principal,Interest,Instalment,Balance
0,-6734.346731,-3516.666667,-10251.013398,393265.653269
1,-6793.552863,-3457.460535,-10251.013398,386472.100406
2,-6853.279515,-3397.733883,-10251.013398,379618.820892
3,-6913.531264,-3337.482134,-10251.013398,372705.289628
4,-6974.312726,-3276.700671,-10251.013398,365730.976901


In [5]:
import matplotlib.pyplot as plt
# gca stands for 'get current axis'
ax = plt.gca()

df.plot(kind='line',x='Principal',y='Interest',color='red',ax=ax)
#df.plot(kind='scatter',x='Principal',y='Balance' ,color='green',ax=ax)
plt.show()


<Figure size 640x480 with 1 Axes>

In [6]:
df.describe()

,Principal,Interest,Instalment,Balance
count,48.000000,48.000000,4.800000e+01,4.800000e+01
mean,-8333.333333,-1917.680064,-1.025101e+04,2.097914e+05
std,1019.710904,1019.710904,7.352954e-12,1.170058e+05
min,-10161.675335,-3516.666667,-1.025101e+04,3.725290e-09
25%,-9168.556777,-2787.109334,-1.025101e+04,1.139545e+05
50%,-8272.457669,-1978.555728,-1.025101e+04,2.167765e+05
75%,-7463.904064,-1082.456621,-1.025101e+04,3.095533e+05
max,-6734.346731,-89.338062,-1.025101e+04,3.932657e+05
